In [ ]:
from chromatic import *
import emcee
import corner
import speclite as speclite
import glob as glob
from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from scipy.optimize import minimize
from scipy.optimize import curve_fit

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

In [ ]:
ordernumbers = np.arange(53,84)
bad_orders = np.array([56,57,59,60,62,63,64,65,66,67,68,69,
                       72,73,74,75,78,79,80,83])
good_orders = np.array([53,54,55,58,61,70,71,76,77,81,82])

# nsteps = 1000
# modeltype = 'Final_Teff_Phot'
# modelname = 'Teff + Phot'
# visits = ['F21','S22']
# orders_I_care_about = [52]

# nsteps = 1000
# modeltype = 'Final_Teff_Spec'
# modelname = 'Teff + Spec'
# visits = ['F21','S22','combined']
# orders_I_care_about = good_orders

# nsteps = 2000
# modeltype = f'Final_Teff_Phot_Spec'
# modelname = 'Teff + Phot + Spec'
# visits = ['F21','S22']
# orders_I_care_about = good_orders

nsteps = 3000
modeltype = f'Final_Teff_Phot_Spec'
modelname = 'Teff + Phot + Spec'
visits = ['combined']
orders_I_care_about = good_orders

for visit in tqdm(visits):
                
    for order in orders_I_care_about:
        label = f'{visit}_{order}_{modeltype}'
        samples_file_label = label+f'_{nsteps}steps'
        '''
        READ IN THE MCMC SAMPLES
        '''
        reader = emcee.backends.HDFBackend(f'../data/samples/{samples_file_label}.h5')
        sampler = reader.get_chain(discard=int(0.25*nsteps), flat=True)
        samples = sampler.reshape((-1, 4)).T
        fspot_sam, dfspot_sam, Tspot_sam, Tamb_sam = samples
        sig1_fspot = np.percentile(fspot_sam, [15.9, 50., 84.1]) # central 1-sigma values
        sig1_dfspot = np.percentile(dfspot_sam, [15.9, 50., 84.1])
        sig1_Tspot = np.percentile(Tspot_sam, [15.9, 50., 84.1])
        sig1_Tamb = np.percentile(Tamb_sam, [15.9, 50., 84.1])
        # Define the 50th percentile and 1-sigma interval parameter values from the samples
        best_params = np.array([sig1_fspot[1], sig1_dfspot[1], sig1_Tspot[1], sig1_Tamb[1]])
        best_params_err_lower = best_params-[sig1_fspot[0], sig1_dfspot[0], sig1_Tspot[0], sig1_Tamb[0]]
        best_params_err_higher = [sig1_fspot[2], sig1_dfspot[2], sig1_Tspot[2], sig1_Tamb[2]]-best_params
        variable_names = [r'f$_{\rm{spot}}$',r'$\Delta$f$_{\rm{spot}}$',r'T$_{\rm{spot}}$',r'T$_{\rm{amb}}$']
        normie_names = ['fspot','dfspot','Tspot','Tamb']

        # Plot samples
        fig, axs = plt.subplots(2,2,figsize=(8,6))
        title_label=f'{visit} {order} {modelname}'
        fig.suptitle(title_label,fontsize=16)
        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[0,1]
        ax4 = axs[1,1]
        colmap = 'Spectral'
        # plot dfspot vs fspot
        ax1.scatter(dfspot_sam,fspot_sam,c=np.log(Tspot_sam/Tamb_sam),
                    cmap=colmap,alpha=0.1,edgecolor=None,vmin=-0.4,vmax=0.4,s=1)
        ax1.plot(dfspot_sam,dfspot_sam)
        ax1.set_ylabel(r'f$_{\rm{spot}}$',fontsize=20)
        ax1.set_ylim(0,0.5)
        ax1.set_xlim(0,0.25)
        # plot dfspot vs log(Tspot/Tamb)
        ax2.scatter(dfspot_sam,np.log(Tspot_sam/Tamb_sam),c=np.log(Tspot_sam/Tamb_sam),
                    cmap=colmap,alpha=0.1,edgecolor=None,vmin=-0.4,vmax=0.4,s=1)
        ax2.set_xlabel(r'$\Delta$ f$_{spot}$',fontsize=20)
        ax2.set_ylabel(r'Log(T$_{\rm{spot}}$/T$_{\rm{amb}}$)',fontsize=20)
        ax2.set_xlim(0,0.25)
        ax2.set_ylim(-.5,0.5)
        # plot fspot vs log(f1), the filling factor of 1 spot
        top_right_plot = ax3.scatter(fspot_sam,np.log(dfspot_sam**2/fspot_sam),
                                        c=np.log(Tspot_sam/Tamb_sam),cmap=colmap,
                                        alpha=0.1,edgecolor=None,vmin=-0.4,vmax=0.4,s=1)
        ax3.plot(np.linspace(0,0.5,100),np.log(np.linspace(0.0001,0.5,100)),label=r'N$_{\rm{spots}}$ = 1')
        ax3.plot(np.linspace(0,0.5,100),np.log(np.linspace(0.0001,0.5,100)/10),label=r'N$_{\rm{spots}}$ = 10')
        ax3.plot(np.linspace(0,0.5,100),np.log(np.linspace(0.0001,0.5,100)/100),label=r'N$_{\rm{spots}}$ = 100')
        ax3.set_xlim(0,0.5)
        ax3.set_ylim(-6,0)
        ax3.set_ylabel(r'Log(f$_1$)',fontsize=20)
        ax3.legend(loc='upper left',frameon=False,fontsize='10')
        fig.colorbar(top_right_plot, ax=axs[:, 1])
        # plot fspot vs log(Tspot/Tamb)
        ax4.scatter(fspot_sam,np.log(Tspot_sam/Tamb_sam),c=np.log(Tspot_sam/Tamb_sam),
                    cmap=colmap,alpha=0.1,edgecolor=None,vmin=-0.4,vmax=0.4,s=1)
        ax4.set_ylim(-.5,0.5)
        ax4.set_xlim(0,0.5)
        ax4.set_xlabel(r'f$_{\rm{spot}}$',fontsize=20)
        # ax4.set_ylabel(r'Log(T$_{\rm{spot}}$/T$_{\rm{amb}}$)',fontsize=20)

        plt.savefig(f'../figs/{visit}_{order}_{modeltype}_samples.png',dpi=200)
        plt.show()
        plt.close()
        
        #make the corner plot
        figure = corner.corner(samples.T, labels=variable_names,
                               quantiles=[0.16, 0.5, 0.84],
                               show_titles=True, title_kwargs={"fontsize": 16})

        # Extract the axes, loop through to set all the limits
        axes = np.array(figure.axes).reshape((4, 4))
        for yi in range(4):
            ax = axes[yi, 0]
            ax.set_xlim(0,0.5)
            ax = axes[yi, 1]
            ax.set_xlim(0,0.5)
            ax = axes[yi, 2]
            ax.set_xlim(2300,5000)
            ax = axes[yi, 3]
            ax.set_xlim(2300,5000)
        plt.savefig(f'../figs/{label}_corner.png',dpi=200)
        plt.show()
        plt.close()